# Part 2 - Data Cleansing

1. Preparation
2. Retrieve the raw data from our SQL-database
3. Pre-analysis of the data and plotting
4. Create SQLite database and insert the clean data into it

## 1. Preparation

In [1]:
#import of the relevant libraries
#without eikon API, since we do not connect to eikon anymore but work with the data from our SQL-database instead

import pandas as pd
pd.set_option("display.max_rows", None, "display.max_columns", None)
import sqlite3
from sqlite3 import Error
import numpy as np
import scipy
from statsmodels.tsa.stattools import adfuller
from statsmodels.regression.rolling import RollingOLS 
import statsmodels.api as sm
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates 
import itertools as it

#### 1.1 - Generate multiple lists for the data under investigation

In [2]:
#generate a list of the cryptocurrencies
rics_crypto = ['BTC=', #Bitcoin
              'ETH=', #Ethereum
              'XRP=', #Ripple
              'LTC=', #Litecoion
              'BCH='] #Bitcoin Cash

#generate a list of the fiat currencies
rics_currency = ['EUR=', #Euro
                 'GBP=', #Pound Sterling
                 'JPY=', #Japanese Yen
                 'CHF=', #Swiss Francs
                 'CAD='] #Canadian Dollar

#generate a list of the commodities
rics_commodities = ['LCOc1', #Crude Oil (ICE Europe Brent Crude Electronic Energy Future)
                    'WTC-', #Crude Oil (WTI Cushing US FOB)
                    'XAU=', #Gold
                    'XAG=', #Silver
                    '.BCOM', #Bloomberg Commodity Index
                    '.dMIWO0EN00PUS'] #MSCI World Energy Index USD (End of Day)

#generate a list of the stock indices
rics_stockindex = ['.NDX', #Nasdaq 100 Index
                   '.SPX', #S&P 500 Index
                   '.FTSE', #FTSE 100 Index
                   '.CSI300', #China Securities Index 300
                   '.dMIWO00000PUS', #MSCI World Price Index USD (End of Day)
                   '.dMIEF00000PUS', #MSCI Emerging Markets Price Index USD (End of Day)
                   '.dMIEU00000PUS'] #MSCI Europe Price Index USD (End of Day)

#### 1.2 - Generate some further lists which will be helpful later on

In [3]:
#generate a list of all rics
rics = rics_crypto + rics_currency + rics_commodities + rics_stockindex

#generate a list of the dependent variables (y)
rics_dependent_variables = rics_crypto + rics_currency

#generate a list of the independent variables (x)
rics_independent_variables = rics_commodities + rics_stockindex

#generate a list of the desired names of the columns
columns =['Bitcoin',
          'Ethereum',
          'Ripple',
          'Litecoin',
          'Bitcoin Cash',
          'EUR',
          'GBP',
          'JPY',
          'CHF',
          'CAD',
          'Crude Oil ICE',
          'Crude Oil WTI',
          'Gold',
          'Silver',
          'Bloomberg Commodity Index',
          'MSCI World Energy',
          'Nasdaq 100',
          'S&P 500',
          'FTSE 100',
          'CSI 300',
          'MSCI World',
          'MSCI Emerging Markets',
          'MSCI Europe']

#generate a list of the desired names of the columns of the dependent variables (crypto and other currencies)
columns_dependent_variables = ['Bitcoin',
                               'Ethereum',
                               'Ripple',
                               'Litecoin',
                               'Bitcoin Cash',
                               'EUR',
                               'GBP',
                               'JPY',
                               'CHF',
                               'CAD']

#generate a list of the desired names of the columns of the independent variables (commodities & indices)
columns_independent_variables = ['Crude Oil ICE',
                               'Crude Oil WTI',
                               'Gold',
                               'Silver',
                               'Bloomberg Commodity Index',
                               'MSCI World Energy',
                               'Nasdaq 100',
                               'S&P 500',
                               'FTSE 100',
                               'CSI 300',
                               'MSCI World',
                               'MSCI Emerging Markets',
                               'MSCI Europe']

#generate a list of the desired start and end date of the analysis
startdate = '2012-01-04'
enddate = '2022-03-31'

## 2. Retrieve the raw data from our SQL-database

In [4]:
#pull the raw data from the SQLite database to python
conn = sqlite3.connect('AQM_Project_Aziz_Oeggerli_Schmid.db')
c = conn.cursor()
c.execute("SELECT * FROM raw_data")
new_data_raw = c.fetchall()


#convert the list to a dataframe using the before defined columns as header
new_data_raw = pd.DataFrame(new_data_raw,columns=['Date','CLOSE','RIC'])


#pivot the tables after RIC and set the date as index --> with the '[rics]' at the end, the order of the columns stays the same
new_data_raw = new_data_raw.pivot(index='Date',columns='RIC',values='CLOSE')[rics]


#change the names of the columns of the dataframe
new_data_raw.columns = columns


#switch format of the index to datetime
new_data_raw.index = pd.to_datetime(new_data_raw.index)

print(new_data_raw.shape)
new_data_raw.head()

(3445, 23)


,Bitcoin,Ethereum,Ripple,Litecoin,Bitcoin Cash,EUR,GBP,JPY,CHF,CAD,Crude Oil ICE,Crude Oil WTI,Gold,Silver,Bloomberg Commodity Index,MSCI World Energy,Nasdaq 100,S&P 500,FTSE 100,CSI 300,MSCI World,MSCI Emerging Markets,MSCI Europe
Date,,,,,,,,,,,,,,,,,,,,,,,
2012-01-04,NaN,NaN,NaN,NaN,NaN,1.2940,1.5615,76.67,0.9412,1.0123,113.70,103.22,1611.54,29.16,144.4944,246.620,2329.71,1277.30,5668.45,2298.753,1203.619572,937.540,1274.260
2012-01-05,NaN,NaN,NaN,NaN,NaN,1.2780,1.5482,77.17,0.9530,1.0197,112.74,101.81,1622.50,29.27,142.3351,244.358,2348.98,1281.06,5624.26,2276.385,1197.031831,932.443,1251.734
2012-01-06,NaN,NaN,NaN,NaN,NaN,1.2720,1.5433,76.98,0.9548,1.0271,113.06,101.56,1616.39,28.70,142.5498,243.602,2356.17,1277.81,5649.68,2290.601,1191.667194,927.061,1244.699
2012-01-09,NaN,NaN,NaN,NaN,NaN,1.2765,1.5457,76.83,0.9496,1.0233,112.45,101.31,1611.00,28.94,143.0837,244.286,2350.65,1280.70,5612.26,2368.570,1192.484691,930.870,1241.264
2012-01-10,NaN,NaN,NaN,NaN,NaN,1.2772,1.5479,76.83,0.9493,1.0168,113.28,102.24,1630.88,29.87,144.2135,246.895,2367.38,1292.08,5696.70,2447.349,1207.553784,948.819,1267.489


## 3. Pre-analysis of the data and plotting

#### 3.1 - We do some pre-analysis to figure out the earliest date on which we have data available for all assets

In [5]:
#in our case the assumption is, that cryptocurrencies have the shortest data availability
#with the following codes we check which cryptocurrency does have the least data availability

# new_data_raw[new_data_raw['Bitcoin'].isnull()].tail(5)
# new_data_raw[new_data_raw['Ethereum'].isnull()].tail(5)
# new_data_raw[new_data_raw['Ripple'].isnull()].tail(5)
# new_data_raw[new_data_raw['Litecoin'].isnull()].tail(5)
# new_data_raw[new_data_raw['Bitcoin Cash'].isnull()].tail(5)


#the result shows, that Ripple does have the least data availability (starting from 04.03.2019)
new_data_raw[new_data_raw['Ripple'].isnull()].tail(5)


#therefore, we can now drop all columns in which Ripple shows a NaN
new_data_raw.dropna(subset=['Ripple'],inplace=True)

In [6]:
#Verification:
new_data_raw.isnull().sum()

#results show, that cryptos do not have any NaN anymore and the rest +/- have a similar amount of NaN
#which indicates that these can mostly be explained by public holidays or weekends
#in a next step, this assumption will be verified by plotting the different columns to conduct

Bitcoin                        0
Ethereum                       0
Ripple                         0
Litecoin                       0
Bitcoin Cash                   0
EUR                          320
GBP                          320
JPY                          320
CHF                          320
CAD                          320
Crude Oil ICE                327
Crude Oil WTI                350
Gold                         321
Silver                       321
Bloomberg Commodity Index    346
MSCI World Energy            320
Nasdaq 100                   346
S&P 500                      346
FTSE 100                     344
CSI 300                      374
MSCI World                   320
MSCI Emerging Markets        320
MSCI Europe                  321
dtype: int64

#### 3.2 - Plotting our data to conduct a visual check and identify potential anomalies in the dataset

In [7]:
#plot and save individual time series --> takes approx. 5-10 minutes
#as already assumed, the charts do not show any anomalies --> therefore, we can go on with the data cleansing

for col in new_data_raw.iteritems(): 
    print('plotting'+col[0])
    fig, ax = plt.subplots()
    indicator=new_data_raw[col[0]]
    ax.plot(indicator, alpha=0.9, color='blue')
    plt.xticks(fontsize=8,rotation=45)
    plt.title('Daily Closing Prices of '+col[0])
    plt.savefig('Charts_data_cleaning/Plot_'+col[0]+'.png')
    plt.close()


#the next and final step would be the fill the NaN due to WE/public holidays with the value of the previous day

plottingBitcoin
plottingEthereum
plottingRipple
plottingLitecoin
plottingBitcoin Cash
plottingEUR
plottingGBP
plottingJPY
plottingCHF
plottingCAD
plottingCrude Oil ICE
plottingCrude Oil WTI
plottingGold
plottingSilver
plottingBloomberg Commodity Index
plottingMSCI World Energy
plottingNasdaq 100
plottingS&P 500
plottingFTSE 100
plottingCSI 300
plottingMSCI World
plottingMSCI Emerging Markets
plottingMSCI Europe


#### 3.3 - The final step of the data cleansing is to fill the remaining NaN

In [8]:
#our previous analysis has shown that the remaining NaN can be lead back to weekends/public holidays on which fiat currencies, commidities and stock indices do not have a price 
#since our data follows a trend, but no seasonality, we can use a linear interpolation in forwarding direction (filling the NaN with the data of the previous day)
all_data_clean = new_data_raw.fillna(axis=0, method='ffill')

#verify that no more NaN are available in the dataset
print(all_data_clean.isnull().sum().sum())
all_data_clean.head()

0


,Bitcoin,Ethereum,Ripple,Litecoin,Bitcoin Cash,EUR,GBP,JPY,CHF,CAD,Crude Oil ICE,Crude Oil WTI,Gold,Silver,Bloomberg Commodity Index,MSCI World Energy,Nasdaq 100,S&P 500,FTSE 100,CSI 300,MSCI World,MSCI Emerging Markets,MSCI Europe
Date,,,,,,,,,,,,,,,,,,,,,,,
2019-03-04,3702.48,125.31,0.30114,45.44,122.91,1.1337,1.3186,111.74,0.9988,1.3302,65.67,56.60,1286.4150,15.0742,80.8184,207.172734,7150.833,2792.81,7134.39,3794.1040,2090.399580,1053.616,1632.467
2019-03-05,3937.60,138.78,0.31261,53.16,130.20,1.1306,1.3175,111.89,1.0040,1.3345,65.86,56.55,1287.1801,15.1222,81.1937,207.120059,7156.785,2789.65,7183.43,3816.0132,2087.502005,1055.117,1632.603
2019-03-06,3945.10,136.35,0.31471,56.39,130.07,1.1305,1.3169,111.75,1.0048,1.3440,65.99,56.22,1286.3600,15.0711,80.7408,205.797671,7112.468,2771.45,7196.00,3848.0903,2078.688801,1055.680,1633.696
2019-03-07,3871.00,141.23,0.31268,56.67,133.83,1.1192,1.3084,111.58,1.0111,1.3453,66.30,56.60,1285.4950,15.0162,80.5200,204.901110,7026.883,2748.93,7157.55,3808.8497,2061.411604,1043.562,1614.594
2019-03-08,3868.30,135.35,0.30762,57.37,128.22,1.1238,1.3015,111.15,1.0078,1.3413,65.74,55.77,1298.0100,15.3100,80.4667,200.989642,7015.690,2743.07,7104.31,3657.5790,2051.123352,1030.127,1600.250


## 4. Create SQLite database and insert the clean data into it

In [9]:
#create a SQLite database
conn = sqlite3.connect('AQM_Project_Aziz_Oeggerli_Schmid.db')
c = conn.cursor()
c.execute('''CREATE TABLE clean_data
         (Date TIMESTAMP,
         CLOSE           INT    NOT NULL,
         RIC            INT     NOT NULL)''')

print("Table created successfully")


#change the names of the columns back to the RIC's (so that we have consistent names in our different SQL databases)
all_data_clean.columns = rics


#switch format of the index to datetime
all_data_clean.index = pd.to_datetime(all_data_clean.index)


#create a new column with the date
#this will later be used to re-genereate the dattime index of the dataframe after pulling them from our SQLite database
index = all_data_clean.index.strftime('%Y-%m-%d')
all_data_clean['Date1'] = index


#transformation into long format and set new column names
all_data_clean_to_insert = pd.melt(all_data_clean, id_vars = ['Date1'], value_vars = all_data_clean.columns.values.tolist()[:-1])
all_data_clean_to_insert.columns = ['Date','RIC','CLOSE'] 


#transform the dataframe into a list
#this needs to be done so that we can insert the data into the before created SQLite database
all_data_clean_to_insert_list = all_data_clean_to_insert.values.tolist()


#upload the data to our SQLite database
c = conn.cursor()
c.executemany("INSERT INTO clean_data(Date, RIC, CLOSE) VALUES (?,?,?)", all_data_clean_to_insert_list)    
conn.commit()

print("Upload to SQL-database successful")

Table created successfully
Upload to SQL-database successful
